# 前準備

privateディレクトリ内にAPI.txt (Google Cloud ConsoleからYouTube Data APIをアクティブにして、認証情報から作成したコード)がある<br>
以下は必要:<br>
pip install google-api-python-client jupyter

## APIキーの読み込み

In [22]:
from googleapiclient.discovery import build
import pandas as pd
from googleapiclient.discovery import build
import json
import re
import random
import string

# APIキーを外部ファイルから読み込む
API_path = "/Users/syokfan/Dropbox/rprojects/youtube_trans/private/API.txt"
with open(API_path, "r") as f:
    API_KEY = f.read().strip()


## 関数の定義

In [39]:
youtube = build("youtube", "v3", developerKey=API_KEY)

def search_videos(query, max_results=5):
    response = youtube.search().list(
        q=query,
        part='snippet',
        type='video',
#        期間を絞りたい場合はここを有効にする
#        publishedAfter='2025-04-01T00:00:00Z',
#        publishedBefore='2025-04-10T00:00:00Z',
#        期間を絞りたい場合はここを有効にする
        maxResults=max_results
    ).execute()

    videos = []
    for item in response['items']:
        video_data = {
            'video_id': item['id']['videoId'],
            'published_at': item['snippet']['publishedAt'],
            'channel_id': item['snippet']['channelId'],
            'channel_title': item['snippet']['channelTitle'],
            'title': item['snippet']['title'],
            'description': item['snippet']['description'],
            'thumbnails': item['snippet']['thumbnails'],
            'publish_time': item['snippet'].get('publishTime'),
        }
        videos.append(video_data)
    return videos

# 統計情報（再生数・高評価数・コメント数など）を取得
def get_video_stats(video_ids):
    response = youtube.videos().list(
        part='statistics,snippet',
        id=','.join(video_ids)
    ).execute()

    stats = {}
    for item in response['items']:
        stats[item['id']] = {
            'viewCount': item['statistics'].get('viewCount'),
            'likeCount': item['statistics'].get('likeCount'),
            'commentCount': item['statistics'].get('commentCount'),
            'favoriteCount': item['statistics'].get('favoriteCount'),
            'tags': item['snippet'].get('tags'),
            'categoryId': item['snippet'].get('categoryId')
        }
    return stats

        
# コメント取得（ユーザー名、投稿時刻、本文、いいね数など）
def get_comments(video_id, max_results=20):
    comments = []
    try:
        response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            textFormat='plainText',
            maxResults=max_results
        ).execute()

        for item in response['items']:
            snippet = item['snippet']['topLevelComment']['snippet']
            comment_data = {
                'author': snippet.get('authorDisplayName'),
                'text': snippet.get('textDisplay'),
                'like_count': snippet.get('likeCount'),
                'published_at': snippet.get('publishedAt')
            }
            comments.append(comment_data)
    except Exception as e:
        print(f"コメント取得エラー: {e}")
    return comments
    
def sanitize_for_variable(name):
    """Pythonの変数名として安全な形式に変換 + ランダムな10桁を追加してユニーク化"""
    name = re.sub(r'\W|^(?=\d)', '_', name)  # 非英数字と先頭の数字をアンダースコアに
    rand_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))
    return f"video_{name}_{rand_suffix}"

## 実行 

queryでキーワードを指定

In [40]:

# 検索ワードを定義
query = "神戸大学"
max_results=30
videos = search_videos(query, max_results)
video_ids = [v['video_id'] for v in videos]
stats = get_video_stats(video_ids)

# 動画情報とコメントを格納するリスト・辞書
video_rows = []
comment_dataframes = {}
recommend_dataframes = {}

for video in videos:
    vid = video['video_id']
    stat = stats.get(vid, {})

    # Python変数名として使える original_id を作成
    original_id = sanitize_for_variable(vid)

    # 動画情報の1行
    video_row = {
        'original_id': original_id,
        'video_id': vid,
        'title': video['title'],
        'channel_title': video['channel_title'],
        'published_at': video['published_at'],
        'description': video['description'],
        'view_count': stat.get('viewCount'),
        'like_count': stat.get('likeCount'),
        'comment_count': stat.get('commentCount'),
        'tags': stat.get('tags')
    }
    video_rows.append(video_row)

    # コメントの取得とDataFrame化
    comments = get_comments(vid)
    if comments:
        comments_df = pd.DataFrame(comments)
        var_name = f"comments_{original_id}"
        globals()[var_name] = comments_df
        comment_dataframes[var_name] = comments_df

# 全動画の情報をDataFrameにまとめる
videos_df = pd.DataFrame(video_rows)

In [41]:
videos_df

,original_id,video_id,title,channel_title,published_at,description,view_count,like_count,comment_count,tags
0,video_H6_rFP95_Lo_wm73m,H6-rFP95-Lo,【神戸大学が謝罪】「5年以下の懲役」の可能性も…旅館で暴れたサークル 学生への処分は？,日テレNEWS,2024-03-25T23:55:37Z,神戸大学のサークルの学生が、旅館で障子や天井などを破壊する迷惑行為をしたことについて、25日...,356452,2584,1906,[社会]
1,video_zXGNtouzhxg_duwym,zXGNtouzhxg,【必見】神戸大が人気な理由は？#武田塾 #大学受験 #参考書 #受験生 #勉強 #受験対策 ...,武田塾チャンネル｜参考書のやり方・大学受験情報,2024-08-06T08:00:36Z,今回は「神戸大が人気な理由は？」について舩澤先生が解説！ 魅力がたくさんの神戸大、気になった...,193354,3465,276,None
2,video_KCkG3tcc29I_z1eba,KCkG3tcc29I,神戸大学と九州大学ってどっちがすごいの？ #shorts,受験コンサル🌐Polaris Academia,2022-12-24T10:00:14Z,逆転合格の勉強法・考え方を配信しています チャンネル登録・高評価よろしくお願いします。 ...,1086542,12047,864,"[ポラリスアカデミア, 神戸大学, 九州大学, 吉村暢浩]"
3,video_AZF6bz4DRcs_nyux5,AZF6bz4DRcs,【神戸大学はやばい？】どれくらい難しい？キャンパス・偏差値・評判など,大学紹介チャンネル,2022-09-14T22:00:05Z,【神戸大学に関するページ】 ○神戸大学の学費：https://www.kobe-u.ac.j...,55032,388,60,"[神戸大学, 神戸大学 キャンパス, 大学紹介チャンネル（せしぶろぐ）]"
4,video__90JtYDS3ZpY_no1gq,90JtYDS3ZpY,【西日本の高校生必見】関関同立か神戸大学を目指す理由を徹底解説,武田塾チャンネル｜参考書のやり方・大学受験情報,2025-01-22T09:00:08Z,西日本の高校生はとりあえず関関同立か神戸大学を目指すべき！という高田先生の持論を今回は関西の...,50573,400,154,"[早稲田, 慶應, 東大, 国公立, march, 大学受験, 参考書, 勉強法, 武田塾,..."
5,video_PhSm75QnYsY_j4vvw,PhSm75QnYsY,旧帝には負けない！神戸大VS旧帝、勝ってる大学はどこ？【wakatte TV】#734,wakatte.tv,2022-06-16T11:00:11Z,世間で高学歴と評される旧帝大 その旧帝大と張り合うと言われている神戸大学の学生たちに、勝って...,646747,6669,2218,"[wakatte.tv, ワカッテ, 武田塾チャンネル, モテる, 塾講師, 大学, 京大,..."
6,video_G00I7t3FiUY_h4zvd,G00I7t3FiUY,どっちがすごい？北海道大学VS神戸大学,CASTDICE TV,2024-05-04T08:00:12Z,個別指導塾CASTDICEでは入塾相談・面談を受け付けています！→ https://cast...,41099,536,404,"[学歴, 東大, CASTDICE TV, 大学受験, 受験勉強, 理系, 勉強法, コバシ..."
7,video_B_vYABQ9D8o_6ck3f,B_vYABQ9D8o,神戸大と早稲田ってどっちが上なんですか？ #shorts,受験コンサル🌐Polaris Academia,2023-02-27T10:00:11Z,逆転合格の勉強法・考え方を配信しています チャンネル登録・高評価よろしくお願いします。 ...,467678,6719,961,"[ポラリスアカデミア, 吉村暢浩, 早稲田大学, 早稲田, 神戸大学]"
8,video_ln8usOE9zVo_anij9,ln8usOE9zVo,【大阪・関西万博】EXPO2025開幕初日散々💨大屋根リング永久保存！大石あきこの地盤沈下は...,777shiryu,2025-04-14T08:00:50Z,関連動画＆プレイリスト EXPO2025 大阪・関西万博 ...,446,None,0,"[大阪・関西万博, 大阪万博, EXPO2025, パビリオン, 大屋根リング, ミャクミャ..."
9,video_xotRHDeVcDY_qh1qk,xotRHDeVcDY,【ほぼ旧帝】神戸大と同じレベルの大学はどこ？#Shorts #神戸 #対決 #関西,wakatte.tv,2025-01-12T11:01:13Z,神戸 #対決 #関西 #大学 #大学受験 #高校生 #バラエティ #偏差値 #学歴 #わかっ...,206358,3864,806,None


In [36]:
comments_video_H6_rFP95_Lo_y3bqbalzx0	

,author,text,like_count,published_at
0,@わじん-d6u,バドミントンをやっている人たちにも失礼じゃないか、、「俺バドミントン部の中学生」,0,2025-03-25T02:44:59Z
1,@katsumi-na1198,サークル名バッドボーイズなら納得の所業だな,1,2025-03-16T05:05:27Z
2,@pashirin,オッサンじゃなくて本人達を出して謝罪させろ！\nほんで、謝罪した直後からヘラヘラ笑ってんな！,0,2025-02-25T12:39:58Z
3,@Abysspeace,日本人は同一視が好きだからこれからは神大卒＝不良とカテゴライズされるだろうね。,0,2025-02-22T11:11:05Z
4,@ひぐらし-y4h,抹籍処分案件だな,0,2025-02-04T08:52:02Z
5,@白米-x4f,これが人間に進化する前の猿か,0,2025-01-11T07:41:59Z
6,@佐藤-u8x,知能どこに行った？？,0,2025-01-06T01:02:52Z
7,@手さげ,貴方達のお家教えて下さい。\n同じ事してあげます。\n(旅館側のきもち),0,2025-01-02T11:28:50Z
8,@uikaatuniv5961,何人かbadboysの子知ってるけどここに書いてる憶測一個も当たってない,1,2024-12-21T12:02:35Z
9,@アメプロ-z7q,副学長心なしかうれしそうに見えるんだが？ 気のせいか？\nてか実名と顔出してもいいだろう…旅...,0,2024-12-02T06:09:55Z


# 気をつけるべきこと (ChatGPT情報)

1. relatedToVideoId の廃止について<br> YouTube Data API v3 の search().list() メソッドにおける relatedToVideoId パラメータは、2023年8月7日をもって廃止されました。 ​ Stack Overflow これにより、特定の動画に関連する動画を直接取得する方法が公式には提供されなくなりました。

2. API の使用量（クォータ）とその増加について<br> YouTube Data API の利用には、API キーごとにクォータ（使用量の上限）が設定されています。​ 🔢 デフォルトのクォータ 通常、1つの Google Cloud プロジェクトには、1日あたり 10,000 ユニットのクォータが割り当てられています。​ 各 API リクエストには、以下のようなユニットが消費されます：​ search.list リクエスト：1ユニット videos.list リクエスト：1ユニット commentThreads.list リクエスト：1ユニット​ Google for Developers Google APIs +6 Stack Overflow +6 Google for Developers +6 つまり、1日に最大で約10,000回のリクエストが可能です。​ 📈 クォータの増加 より多くのリクエストが必要な場合は、Google Cloud Console からクォータの増加を申請することができます。​ 申請には、以下の情報が求められます：​ 使用目的の詳細 必要なクォータ数 予想されるトラフィック量​ 申請が承認されれば、クォータの上限を引き上げることが可能です。

# 実際の研究例

期間（月ごと）を区切ってトップ100に入る動画を検討<br>
https://journals.sagepub.com/doi/full/10.1177/1940161220964767
